# TP de NLP sur la Séance 3 


## Bibliothèques 

In [3]:
# Installation 

#!pip install kaggle
#!kaggle datasets download -d Cornell-University/movie-dialog-corpus


Dataset URL: https://www.kaggle.com/datasets/Cornell-University/movie-dialog-corpus


  0%|          | 0.00/8.91M [00:00<?, ?B/s]
 11%|█         | 1.00M/8.91M [00:00<00:04, 1.98MB/s]
 34%|███▎      | 3.00M/8.91M [00:00<00:01, 5.41MB/s]
 56%|█████▌    | 5.00M/8.91M [00:00<00:00, 8.17MB/s]
 79%|███████▊  | 7.00M/8.91M [00:00<00:00, 9.61MB/s]
100%|██████████| 8.91M/8.91M [00:01<00:00, 10.4MB/s]
100%|██████████| 8.91M/8.91M [00:01<00:00, 8.13MB/s]



License(s): other



In [4]:
#Import des bibliothèques 

#traitement du csv
import os
import csv

# Traitement du NLP
import spacy

# Création du modèle word to vect 
import gensim

#bibliothèques usuelles
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# PCA 
from sklearn.manifold import TSNE

#Ajout d'une barre d'avancement
from tqdm.notebook import tqdm

## Declaration des classes 

In [25]:
# définir une classe
class Corpus(object):
    def __init__(self, filename):
        self.filename = filename
        self.nlp = spacy.blank("en")
    
    # méthode qui itère sur les instances de la classe
    def __iter__(self):
        with open(self.filename, "r") as i:
            reader = csv.reader(i, delimiter=",")
            for _, abstract in reader:
                tokens = [t.text.lower() for t in self.nlp(abstract)]
                yield tokens

## Import traitement et décompression du fichier zip

In [24]:
#À compiler dans le bash 

#mkdir data/movie-dialog-corpus/
#unzip -o movie-dialog-corpus.zip -d data/movie-dialog-corpus/

## Traitement des données

**Convertir le tsv en csv**

In [29]:
# Chemin du répertoire contenant les fichiers TSV
tsv_directory = "data/movie-dialog-corpus/"
# Chemin du répertoire où les fichiers CSV convertis seront enregistrés
csv_directory = "data/movie-dialog-csv/"

# Création du répertoire de destination s'il n'existe pas
os.makedirs(csv_directory, exist_ok=True)

# Parcours de tous les fichiers dans le répertoire TSV
for file_name in os.listdir(tsv_directory):
    if file_name.endswith('.tsv'):
        # Chemin complet du fichier TSV
        tsv_file_path = os.path.join(tsv_directory, file_name)
        # Chemin complet du fichier CSV de destination
        csv_file_path = os.path.join(csv_directory, os.path.splitext(file_name)[0] + '.csv')

        # Lecture du fichier TSV et écriture dans le fichier CSV
        with open(tsv_file_path, 'r', encoding='utf-8') as tsv_file, \
             open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:

            # Création d'un objet lecteur TSV
            tsv_reader = csv.reader(tsv_file, delimiter='\t')
            # Création d'un objet écrivain CSV
            csv_writer = csv.writer(csv_file)

            # Copie des lignes du fichier TSV dans le fichier CSV
            for row in tsv_reader:
                csv_writer.writerow(row)

In [36]:
documents = Corpus("data/movie-dialog-csv/movie_conversations.csv")

In [40]:
with open("data/movie-dialog-csv/movie_conversations.csv",'r',encoding='utf-8') as file:
    documents = [line.strip().split() for line in file]

## Entrainement des word embedings

In [42]:
import gensim
from gensim.models import Word2Vec
import csv
import spacy

# Chargement du modèle SpaCy pour le traitement du langage naturel
nlp = spacy.load("en_core_web_sm")

# Liste pour stocker les phrases prétraitées
preprocessed_sentences = []

# Chargement des données à partir du fichier CSV
with open("data/movie-dialog-csv/movie_conversations.csv", 'r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        # Prétraitement de chaque ligne du fichier CSV
        # (vous pouvez personnaliser cette étape en fonction de vos besoins)
        tokens = [token.text.lower() for token in nlp(row[0])]
        preprocessed_sentences.append(tokens)

# Construction et entraînement du modèle Word2Vec
model = Word2Vec(sentences=preprocessed_sentences,
                 min_count=100,  # Ignorer les mots ayant une fréquence inférieure à 100
                 window=5,       # Taille de la fenêtre contextuelle
                 vector_size=100)  # Dimension des vecteurs de mots

# Vous pouvez maintenant utiliser le modèle pour effectuer des tâches de traitement du langage naturel


## Representation des phrases 

In [43]:
model.wv['Transformer'] # afficher le vecteur du mot 'language'

KeyError: "Key 'Transformer' not present"

## Calcul des similarités 

In [ ]:
print(model.wv.similarity('Transformer', 'language')) # 100% de similarité

In [ ]:
model.wv.similar_by_word('NMT',topn=15) # afficher les mots similaires à 'language'

In [ ]:
print(model.wv.doesnt_match("bert word2vec GPT Transformer".split()))

## Explorations des données 

In [ ]:
#visualisation du word embedding dans l'espace 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.manifold import TSNE # importer TSNE pour visualiser les vecteurs dans un espace 2D

#on choisis un mot et on affiche les mots similaires
target_word = 'BERT'
similar_words = [word for word, _ in model.wv.most_similar(target_word, topn=200)] + [target_word] # on prend les 20 mots les plus similaires à 'BERT' et on ajoute 'BERT' à la liste

# on fait le embedding des mots similaires
embeddings = [model.wv[word] for word in similar_words] + model.wv[target_word] 
# on ajoute le vecteur de 'BERT' à la liste des vecteurs des mots similaires
mapped_embeddings = TSNE(n_components=2, # on veut réduire la dimensionalité à 2D
                         metric='cosine', # on utilise la distance cosinus pour mesurer la similarité entre les vecteurs
                         init='pca', # on initialise les vecteurs avec PCA
                         perplexity=10, # on définit la perplexité à 10 pour que les points soient bien répartis dans l'espace, 10 veut dire qu'on regarde les 10 plus proches voisins
                         ).fit_transform(embeddings)


## Visualisation des résultats 

In [ ]:
plt.figure(figsize=(20, 20)) # on définit la taille de la figure
x = mapped_embeddings[:, 0] # on prend la première colonne de la matrice
y = mapped_embeddings[:, 1] # on prend la deuxième colonne de la matrice
plt.scatter(x, y) # on affiche les points
for i, word in enumerate(similar_words): # on affiche les mots similaires à 'BERT'
    plt.annotate(word, (x[i], y[i]), fontsize=12) # on affiche le mot à la position correspondante
plt.show() # on affiche la figure

## Analyse des résultats 

In [ ]:
from tqdm import tqdm # importer tqdm pour afficher une barre de progression

nlp= spacy.load('en_core_web_sm') # charger le modèle de langue anglaise

wordic= {} # initialiser un dictionnaire vide
for word in tqdm(model.wv.index_to_key): # pour chaque mot dans le vocabulaire du modèle Word2Vec
    wordic[word]= nlp(word)[0].pos_ # ajouter le mot et son vocabulaire dans le dictionnaire

In [ ]:
wordic['translation']